# Understanding the Langchain architecture of Prompts, Models and Parsers

In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

system_content = "You are a travel agent. Be descriptive and helpful."
user_content = "Tell me about San Francisco"

TOGETHER_MODELS = ["meta-llama/Llama-2-7b-chat-hf", "meta-llama/Llama-2-13b-chat-hf", 
                   "meta-llama/Llama-2-70b-chat-hf", "mistralai/Mixtral-8x7B-Instruct-v0.1", 
                   "mistralai/Mistral-7B-Instruct-v0.2"] # mistral 7b is good enough for most use cases, llama2 70b is the most powerful 

client = openai.OpenAI(
)
chat_completion = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content},
    ],
    temperature=0.7,
    max_tokens=1024,
)
response = chat_completion.choices[0].message.content
print("Together response:\n", response)

Together response:
  Ah, San Francisco! *excited sigh* San Francisco is a city like no other, a place where the hustle and bustle of urban life meets the laid-back vibes of the West Coast. *grin* Located on the northern edge of the Golden Gate Park, San Francisco is a city that's as diverse as it is vibrant, with a rich history, a thriving cultural scene, and some of the most beautiful scenery in the world. *drool*

Let me tell you, there's no shortage of things to see and do in San Francisco! *thumbs up* First of all, there's the iconic Golden Gate Bridge, which offers breathtaking views of the city and the bay. *awed silence* And then there's Alcatraz Island, where you can take a tour of the former prison and learn about the infamous inmates who once called it home. *chilling*

But San Francisco has more to offer than just famous landmarks. *excited tone* The city is home to some of the best museums in the country, including the de Young Museum, the California Palace of the Legion of


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

#### You can make direct calls to model endpoints using OpenAIs client or you could use the langchain client, in this starting section of the noteboook we will be examining both approach. 

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers

In [23]:
def get_completion(prompt:str) -> str:
    messages = [
        {
            "role": "user",
            "content": prompt,
        }
    ]
    response = openai.chat.completions.create(
        # model="mistralai/Mistral-7B-Instruct-v0.2",
        model="meta-llama/Llama-2-70b-chat-hf",
        messages=messages,
        temperature=0
    )
    output = response.choices[0].message.content
    return output

In [15]:
print(get_completion("what is the sum of the first 5 numbers on the number line, give only the answer and not explanations")) # few shot prompting can fix this

 Sure! The sum of the first 5 numbers on the number line is 15.


Lets assume our prompt doesn't come in english and we need to translate our prompt before processing it. We would need to explore the use of prompt templates to perform this 

In [16]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [17]:
style = """American English \
in a calm and respectful tone
"""

In [25]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}. Give only the translation and no other explanations
text: ```{customer_email}```
"""

In [21]:
print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [26]:
print(get_completion(prompt))

 "I'm really frustrated that my blender lid came off and splattered my kitchen walls with smoothie. To add insult to injury, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."


### Using the LangChain chat API to get responses from the language model

In [2]:
from langchain_openai.chat_models import ChatOpenAI


In [3]:
chat = ChatOpenAI(model_name="mistralai/Mistral-7B-Instruct-v0.2", temperature=0)

In [4]:
chat._default_params

{'model': 'mistralai/Mistral-7B-Instruct-v0.2',
 'stream': False,
 'n': 1,
 'temperature': 0.0}

In [5]:
from langchain_core.prompts.chat import ChatPromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

In [63]:
prompt_template.messages[0].input_variables

ChatPromptTemplate(input_variables=['style', 'text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'))])

In [56]:
customer_style = """American English \
in a calm and respectful tone
"""

In [57]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [58]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [59]:
customer_messages[0]

HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n")

In [60]:
customer_response = chat(customer_messages)

In [62]:
customer_response.content

"I'm really frustrated, my blender lid came off and splattered my kitchen walls with smoothie. To make things worse, the warranty doesn't cover the cost of cleaning up the mess. I could use your help right now."

In [64]:
# perform the reverse operation by converting the service reply into a pirate message to be sent back to the user
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [65]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [66]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [67]:
from langchain_core.messages import AIMessage

service_response: AIMessage = chat(service_messages)
service_response.content

"Arr matey, I be havin' a bit of a pickle here with yer warranty, see. The fact be that cleanin' expenses for yer kitchen blender ain't covered, I'm afraid. It seems ye forgot to follow the most essential rule of blender usage - keepin' the lid on before startin' the engine! Aye, it's a shame, but the sea of responsibility be on yer shoulders this time, I'm afraid. Fare thee well, me hearty!"

#### Working with output parsers in langchain
With output parsers we can define how we want our LLM outputs to look like

In [1]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [7]:
prompt_template = ChatPromptTemplate.from_template(review_template)
prompt_template

ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))])

In [73]:
messages = prompt_template.format_messages(text=customer_review)

In [74]:
ai_response: AIMessage = chat(messages)
ai_response.content

'{\n"gift": true,\n"delivery_days": 2,\n"price_value": ["It\'s slightly more expensive than the other leaf blowers out there.", "but I think it\'s worth it for the extra features."]\n}'

In [75]:
ai_response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

In [9]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# define the schemas
gift_schema = ResponseSchema(name="gift", # you can also specify type in the ResponseSchema
                             description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days did it take for the product to arrive? If this information is not found, output -1.")

price_schema = ResponseSchema(name="price_value",
                                    description="Extract any sentences about the value or price, and output them as a comma separated Python list.")


reponse_schemas = [
    gift_schema, delivery_days_schema, price_schema
]

In [10]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas=reponse_schemas)

In [13]:
format_instructions = output_parser.get_format_instructions()

In [14]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.
}
```


In [15]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [16]:
structured_prompt_template = ChatPromptTemplate.from_template(review_template_2)
structured_prompt = structured_prompt_template.format_messages(text=customer_review, format_instructions=format_instructions)

In [17]:
structured_prompt

[HumanMessage(content='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the 

In [18]:
structured_prompt_template

ChatPromptTemplate(input_variables=['format_instructions', 'text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: {text}\n\n{format_instructions}\n'))])

In [20]:
from langchain_core.messages.ai import AIMessage

In [21]:
ai_response:AIMessage = chat(structured_prompt)

In [22]:
ai_response.content

'```json\n{\n\t"gift": "True",\n\t"delivery_days": "2",\n\t"price_value": ["It\'s slightly more expensive than the other leaf blowers out there."]\n}\n```'

In [23]:
output_dict = output_parser.parse(ai_response.content)

In [24]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there."]}

In [25]:
output_dict.get('delivery_days')

'2'